In [1]:
from pathlib import Path
from pprint import pprint

import pandas as pd
import numpy as np

from bayes_opt import BayesianOptimization
from isic_helper import compute_pauc, compute_auc

In [2]:
def blend_optimizer(oof_preds_df, oof_columns, folds, init_points = 20, n_iter = 100):
    pbounds = {f"w{i}": (1e-5, 1.0) for i in range(len(oof_columns))}

    def dim_opt(oof_preds_df, *args):
        weights = args
        score = 0
        for fold in folds:
            fold_ensemble_preds = 0
            for weight, oof_column in zip(weights, oof_columns):
                fold_ensemble_preds += weight * oof_preds_df.loc[oof_preds_df["fold"] == fold, oof_column].rank(pct=True).values
            score += compute_pauc(np.array(oof_preds_df.loc[oof_preds_df["fold"] == fold, "target"]), fold_ensemble_preds)
        return score / len(folds)

    def q(**ws):
        ws = tuple(ws.values())
        return dim_opt(oof_preds_df, *ws)

    optimizer = BayesianOptimization(
        f=q,
        pbounds=pbounds,
        random_state=2022,
    )

    optimizer.maximize(
        init_points=init_points,
        n_iter=n_iter,
    )
    
    weights = []
    for i in range(len(oof_columns)):
        weights.append(optimizer.max["params"][f"w{i}"])
    
    print(f"Best pAUC: {optimizer.max['target']}")
    print(f"Best weights: {weights}")
    return weights

In [3]:
id_column = "isic_id"
target_column = "target"
ensemble_column = "oof_ensemble"
fold_column = "gkf_fold"

boosting_model_names = ["xgb", "xgb", "lgb", "cb"]
boosting_versions = ["v1", "v4", "v6", "v1"]
boosting_modes = ["train", "train", "train", "train"]
boosting_oof_columns = [f"oof_{model_name}_{version}" for model_name, version in zip(boosting_model_names, boosting_versions)]

cnn_model_names = ["efficientnet_b0", "efficientnet_b1", "efficientnet_b2"]
cnn_versions = ["v1", "v1", "v1"]
cnn_modes = ["trainbinary", "trainbinary", "trainbinary"]
cnn_oof_columns = [f"oof_{model_name}_{version}" for model_name, version in zip(cnn_model_names, cnn_versions)]

model_names = boosting_model_names + cnn_model_names
versions = boosting_versions + cnn_versions
modes = boosting_modes + cnn_modes
oof_columns = boosting_oof_columns + cnn_oof_columns

paths = [f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-{mode}" for model_name, version, mode in zip(model_names, versions, modes)]

val_auc_scores = {}
val_pauc_scores = {}
for idx, path in enumerate(paths):
    model_name = model_names[idx]
    version = versions[idx]
    mode = modes[idx]
    model_identifier = f"{model_name}_{version}"
    oof_preds_model_df = pd.read_csv(f"{path}/oof_train_preds_{model_identifier}.csv")
    if idx == 0:
        oof_preds_df = oof_preds_model_df.copy()
    else:
        oof_preds_df = oof_preds_df.merge(oof_preds_model_df[[id_column, f"oof_{model_name}_{version}"]],
                                          on="isic_id", how="inner")
        assert oof_preds_df.shape[0] == oof_preds_model_df.shape[0]
    
    val_auc_scores[f"{model_name}_{version}"] = {}
    val_pauc_scores[f"{model_name}_{version}"] = {}

all_folds = np.unique(oof_preds_df["fold"])
weights = blend_optimizer(
    oof_preds_df, boosting_oof_columns + cnn_oof_columns, all_folds,
    init_points=50, n_iter=100
)

|   iter    |  target   |    w0     |    w1     |    w2     |    w3     |    w4     |    w5     |    w6     |
-------------------------------------------------------------------------------------------------------------
| 1         | 0.18      | 0.009369  | 0.4991    | 0.1134    | 0.04998   | 0.6854    | 0.487     | 0.8977    |
| 2         | 0.1833    | 0.6475    | 0.897     | 0.7211    | 0.8314    | 0.8276    | 0.8336    | 0.957     |
| 3         | 0.1821    | 0.3681    | 0.4948    | 0.3395    | 0.6194    | 0.9775    | 0.09644   | 0.7442    |
| 4         | 0.1827    | 0.2925    | 0.2987    | 0.7525    | 0.01867   | 0.5237    | 0.8644    | 0.3888    |
| 5         | 0.1817    | 0.2122    | 0.4752    | 0.5647    | 0.3494    | 0.9759    | 0.03783   | 0.7943    |
| 6         | 0.1826    | 0.3579    | 0.748     | 0.9145    | 0.3727    | 0.9649    | 0.08139   | 0.04246   |
| 7         | 0.1832    | 0.2968    | 0.3637    | 0.4903    | 0.6685    | 0.6734    | 0.5721    | 0.0806    |
| 8       

In [4]:
weights

[1e-05,
 0.5438482601110255,
 0.4926535436501769,
 0.8855971967954079,
 1e-05,
 0.38632196093261234,
 0.1559393297164371]

In [5]:
val_auc_scores["ensemble"] = {}
val_pauc_scores["ensemble"] = {}
for fold in all_folds:
    fold_index = oof_preds_df[oof_preds_df["fold"] == fold].index
    fold_target = oof_preds_df.loc[fold_index, target_column]
    fold_ensemble_preds = 0
    for model_name, version, weight in zip(model_names, versions, weights):
        fold_model_preds = oof_preds_df.loc[fold_index, f"oof_{model_name}_{version}"]
        fold_ensemble_preds += fold_model_preds.rank(pct=True).values * weight 
        
        val_auc_scores[f"{model_name}_{version}"][f"fold_{fold}"] = compute_auc(fold_target, fold_model_preds)
        val_pauc_scores[f"{model_name}_{version}"][f"fold_{fold}"] = compute_pauc(fold_target, fold_model_preds, min_tpr=0.8)
    
    oof_preds_df.loc[fold_index, ensemble_column] = fold_ensemble_preds
    val_auc_scores["ensemble"][f"fold_{fold}"] = compute_auc(fold_target, fold_ensemble_preds)
    val_pauc_scores["ensemble"][f"fold_{fold}"] = compute_pauc(fold_target, fold_ensemble_preds, min_tpr=0.8)

for model_name, version, weight in zip(model_names, versions, weights):
    print(f"Model: {model_name}_{version} | Weightage: {weight}")
    
    print("Val AUC scores:")
    pprint(val_auc_scores[f"{model_name}_{version}"])
    print("Val PAUC scores:")
    pprint(val_pauc_scores[f"{model_name}_{version}"])
    
    cv_model_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[f"oof_{model_name}_{version}"])
    cv_model_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[f"oof_{model_name}_{version}"], min_tpr=0.8)

    cv_model_auc_avg = np.mean(list(val_auc_scores[f"{model_name}_{version}"].values()))
    cv_model_pauc_avg = np.mean(list(val_pauc_scores[f"{model_name}_{version}"].values()))

    cv_model_auc_std = np.std(list(val_auc_scores[f"{model_name}_{version}"].values()))
    cv_model_pauc_std = np.std(list(val_pauc_scores[f"{model_name}_{version}"].values()))
    
    print(f"CV AUC OOF: {cv_model_auc_oof}")
    print(f"CV PAUC OOF: {cv_model_pauc_oof}")
    print(f"CV AUC AVG: {cv_model_auc_avg}")
    print(f"CV PAUC AVG: {cv_model_pauc_avg}")
    print(f"CV AUC STD: {cv_model_auc_std}")
    print(f"CV PAUC STD: {cv_model_pauc_std}")
    print("\n")

print("Val AUC scores:")
pprint(val_auc_scores["ensemble"])
print("Val PAUC scores:")
pprint(val_pauc_scores["ensemble"])

cv_ensemble_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[ensemble_column])
cv_ensemble_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[ensemble_column], min_tpr=0.8)

cv_ensemble_auc_avg = np.mean(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_avg = np.mean(list(val_pauc_scores["ensemble"].values()))

cv_ensemble_auc_std = np.std(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_std = np.std(list(val_pauc_scores["ensemble"].values()))

print(f"CV AUC OOF: {cv_ensemble_auc_oof}")
print(f"CV PAUC OOF: {cv_ensemble_pauc_oof}")
print(f"CV AUC AVG: {cv_ensemble_auc_avg}")
print(f"CV PAUC AVG: {cv_ensemble_pauc_avg}")
print(f"CV AUC STD: {cv_ensemble_auc_std}")
print(f"CV PAUC STD: {cv_ensemble_pauc_std}")
print(f"CV PAUC RANGE: ({cv_ensemble_pauc_avg - cv_ensemble_pauc_std}, {cv_ensemble_pauc_avg + cv_ensemble_pauc_std})")

Model: xgb_v1 | Weightage: 1e-05
Val AUC scores:
{'fold_1': 0.9872585081165387,
 'fold_2': 0.9735715113083022,
 'fold_3': 0.9619627547391147,
 'fold_4': 0.9797380227377669,
 'fold_5': 0.9801858256801913}
Val PAUC scores:
{'fold_1': 0.19054790748764813,
 'fold_2': 0.178681349764785,
 'fold_3': 0.1680910009858856,
 'fold_4': 0.1826555889730566,
 'fold_5': 0.18326445745486586}
CV AUC OOF: 0.9729932042284457
CV PAUC OOF: 0.17698608915392508
CV AUC AVG: 0.9765433245163827
CV PAUC AVG: 0.18064806093324823
CV AUC STD: 0.008481907330460713
CV PAUC STD: 0.007354323473052156


Model: xgb_v4 | Weightage: 0.5438482601110255
Val AUC scores:
{'fold_1': 0.9774012004442393,
 'fold_2': 0.9585113204613043,
 'fold_3': 0.9635616209706606,
 'fold_4': 0.9713384957132696,
 'fold_5': 0.9692148821814502}
Val PAUC scores:
{'fold_1': 0.18439364147339235,
 'fold_2': 0.16635088150424956,
 'fold_3': 0.17198002333678597,
 'fold_4': 0.174545119866219,
 'fold_5': 0.17352956799065142}
CV AUC OOF: 0.947856723136131
CV P

In [6]:
oof_preds_df.pivot_table(index="target", values=oof_columns + [ensemble_column], aggfunc="mean")

,oof_cb_v1,oof_efficientnet_b0_v1,oof_efficientnet_b1_v1,oof_efficientnet_b2_v1,oof_ensemble,oof_lgb_v6,oof_xgb_v1,oof_xgb_v4
target,,,,,,,,
0,0.009339,0.002347,0.000992,0.000783,1.231063,0.010528,0.008914,0.014427
1,0.423538,0.289894,0.230465,0.211758,2.396959,0.455325,0.412346,0.329018


In [7]:
oof_preds_df[oof_columns + [ensemble_column]].corr()

,oof_xgb_v1,oof_xgb_v4,oof_lgb_v6,oof_cb_v1,oof_efficientnet_b0_v1,oof_efficientnet_b1_v1,oof_efficientnet_b2_v1,oof_ensemble
oof_xgb_v1,1.000000,0.717820,0.957394,0.931184,0.547011,0.448652,0.407370,0.324806
oof_xgb_v4,0.717820,1.000000,0.711025,0.674799,0.530451,0.403045,0.387978,0.321788
oof_lgb_v6,0.957394,0.711025,1.000000,0.927002,0.558186,0.458106,0.422038,0.351586
oof_cb_v1,0.931184,0.674799,0.927002,1.000000,0.581616,0.495410,0.454357,0.341369
oof_efficientnet_b0_v1,0.547011,0.530451,0.558186,0.581616,1.000000,0.779515,0.783158,0.141168
oof_efficientnet_b1_v1,0.448652,0.403045,0.458106,0.495410,0.779515,1.000000,0.890838,0.088755
oof_efficientnet_b2_v1,0.407370,0.387978,0.422038,0.454357,0.783158,0.890838,1.000000,0.080359
oof_ensemble,0.324806,0.321788,0.351586,0.341369,0.141168,0.088755,0.080359,1.000000
